In [23]:
import onnx
import numpy as np
from onnx import numpy_helper
from tabulate import tabulate
import os
import math
import struct

# Скрипты

In [24]:
def toString(indata, separate = ", "):
  '''
  Перевод спсика, кортежа или словаря в строку.
  :param indata: Данные для трансформации
  '''
  if isinstance(indata, (list, tuple)):
    return separate.join([toString(value).rstrip("\n") for value in indata])
  if isinstance(indata, dict):
    st = ""
    for key, value in indata.items():
      st += "{}: {}\n".format(key, value)
    return st
  return str(indata)

In [25]:
def toHex(value, tp = 'int', bits = 8):
    s = ''
    if (tp == 'int'):
        bits = math.ceil(bits / 4.0)
        mx = int('0x' + 'f' * bits, 16)
        s = hex(value & mx)[2:]
        if (len(s) < bits):
            s = '0' * (bits - len(s)) + s
    if (tp == 'float'):
        if (bits == 64):
            s = hex(struct.unpack('<Q', struct.pack('<d', value))[0])[2:]
        else:
            s = hex(struct.unpack('<I', struct.pack('<f', value))[0])[2:]

        bits = math.ceil(bits / 4.0)
        if (len(s) < bits):
            s = '0' * (bits - len(s)) + s
    return s.upper()

# Имена весов

In [26]:
def getWeightsName(model):
    '''
    Возвращает имена весов
    :param model: открытая onnx модель
    :return: список имен слоев
    '''
    layers = []
    weights = model.graph.initializer
    for i in range(len(weights)):
        layers.append(weights[i].name)
    return layers

# Имена слоев

In [27]:
def getLayersName(model):
    '''
    Возвращает имена слои нейронной сети. Работает не для каждой модели!!!
    :param model: открытая onnx модель
    :return: список имен слоев модели в порядке продвижения данных
    '''
    layers = []
    nodes = model.graph.node
    for i in range(len(nodes)):
        layers.append(nodes[i].name)
    return layers

# Получение весов

In [28]:
def getWeightValue(model, weightName):
    weights = model.graph.initializer
    for i in range(len(weights)):
        if (weights[i].name == weightName):
            return numpy_helper.to_array(weights[i])
    return None

# Вывод схемы нейронной сети

In [29]:
def printScheme(model):
    '''
    Печатает схему работы нейронной сети. Работает не для каждой модели!!!
    :param model: открытая onnx модель
    :return:
    '''
    nodes = model.graph.node
    first = model.graph.input
    last = model.graph.output
    print('Model info:')
    for i in range(len(first)):
        print('\t{:>10s}_{}:'.format('Input', i))
        print('\t\t{:>10s}: {:<50s}'.format('name', toString(first[i].name)))
        print('\t\t{:>10s}: {:<50s}'.format('shape', '[' + toString(
            [str(v).split(': ')[-1].rstrip("\n") for v in list(first[i].type.tensor_type.shape.dim)]) + ']'))
    print('Список слоев:')
    for i in range(len(last)):
        print('\t{:>10s}_{}:'.format('Output', i))
        print('\t\t{:>10s}: {:<50s}'.format('name', toString(last[i].name)))
        print('\t\t{:>10s}: {:<50s}'.format('shape', '[' + toString(
            [str(v).split(': ')[-1].rstrip("\n") for v in list(last[i].type.tensor_type.shape.dim)]) + ']'))

    for i in range(len(nodes)):
        print('{:>15}: {:^50} --> {:^50}'.format(toString(nodes[i].name), toString(list(nodes[i].input)),
                                                 toString(list(nodes[i].output))))


In [30]:
def printSchemeTable(model):
    '''
    Печатает схему работы нейронной сети. Работает не для каждой модели!!!
    :param model: открытая onnx модель
    :return:
    '''
    nodes = model.graph.node

    column_list = ["Name", "Inputs", "Outputs"]
    first = model.graph.input
    last = model.graph.output
    print('Model info:')
    for i in range(len(first)):
        print('\t{:>10s}_{}:'.format('Input', i))
        print('\t\t{:>10s}: {:<50s}'.format('name', toString(first[i].name)))
        print('\t\t{:>10s}: {:<50s}'.format('shape', '[' + toString([str(v).split(': ')[-1].rstrip("\n") for v in list(first[i].type.tensor_type.shape.dim)]) + ']'))

    for i in range(len(last)):
        print('\t{:>10s}_{}:'.format('Output', i))
        print('\t\t{:>10s}: {:<50s}'.format('name', toString(last[i].name)))
        print('\t\t{:>10s}: {:<50s}'.format('shape', '[' + toString([str(v).split(': ')[-1].rstrip("\n") for v in list(last[i].type.tensor_type.shape.dim)]) + ']'))

    print('Список слоев:')
    value_list = []
    for i in range(len(nodes)):
        value_list.append([toString(nodes[i].name), toString(list(nodes[i].input)), toString(list(nodes[i].output))])

    print(tabulate(value_list, column_list, tablefmt="grid"))

# Вывод информации о весах

In [31]:
def printWieghtsInfo(model):
    '''
    Печатает информацию о весах.
    :param model: открытая onnx модель
    :return:
    '''
    weights = model.graph.initializer
    column_list = ["Name", 'Shape']
    print('Список весов:')
    value_list = []
    for i in range(len(weights)):
        value_list.append([toString(weights[i].name), toString(weights[i].dims)])

    print(tabulate(value_list, column_list, tablefmt="grid"))

# Сохоранение всех весов модели

In [32]:
def saveAllWeights(model, folder):
    folder = folder + '_raw'
    if (not os.path.exists(folder)):
        os.mkdir(folder)

    weights = model.graph.initializer
    for i in range(len(weights)):
        f = open(folder + '\\' + toString(weights[i].name) + '.txt', 'w')
        f.write(toString(numpy_helper.to_array(weights[i])))
        f.close()

In [33]:
def saveAllWeightsHex(model, folder, weightsType = 'int8', scale = 1):
    weights = model.graph.initializer

    folder = folder + '_hex'
    if (not os.path.exists(folder)):
        os.mkdir(folder)

    tp = ''.join(c for c in weightsType if not c.isdigit())
    bits = int(''.join(c for c in weightsType if c.isdigit()))
    for i in range(len(weights)):
        weight = numpy_helper.to_array(weights[i])
        shapeLen = len(weight.shape)
        if (shapeLen <= 2):
            fileName = toString(weights[i].name) + '.hex'
            for c in '\\/:*?\"<>|':
                fileName = fileName.replace(c, '.')
            f = open(folder + '\\' + fileName, 'w')

            w1 = weight
            ws = w1.shape
            w2 = w1.reshape(np.array(ws).prod())
            w2 = np.array([toHex(int(v * scale), tp, bits) for v in w2])
            w3 = w2.reshape(ws)
            for q in w3:
                if (shapeLen == 1):
                    f.write(q + ' ')
                else:
                    for w in q:
                        f.write(w + ' ')
                    f.write('\n')

            f.close()

In [34]:
# Задание настроек

In [35]:
folder = 'take_layers'
modelName = 'test2.onnx'

In [36]:
model = onnx.load(folder + '\\' + modelName)

In [37]:
printWieghtsInfo(model)

Список весов:
+-------------------+-------------+
| Name              | Shape       |
+===================+=============+
| dense_12/kernel:0 | [1024, 784] |
+-------------------+-------------+
| dense_12/bias:0   | [784]       |
+-------------------+-------------+
| dense_11/kernel:0 | [512, 1024] |
+-------------------+-------------+
| dense_11/bias:0   | [1024]      |
+-------------------+-------------+
| dense_10/kernel:0 | [256, 512]  |
+-------------------+-------------+
| dense_10/bias:0   | [512]       |
+-------------------+-------------+
| dense_9/kernel:0  | [100, 256]  |
+-------------------+-------------+
| dense_9/bias:0    | [256]       |
+-------------------+-------------+


In [38]:
saveAllWeightsHex(model=model,
                  folder=folder + '\\weights_' + '.'.join(modelName.split('.')[:-1]),
                  weightsType='float32',
                  scale=100)